In [14]:
import os

In [15]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Chicken-Disease-Classification-EndtoEnd-DLProject\\research'

In [16]:
os.chdir("../")

In [17]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Chicken-Disease-Classification-EndtoEnd-DLProject'

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [19]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [21]:
import time


In [22]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [23]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [34]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        # Clear backend to reset any pre-existing states
        tf.keras.backend.clear_session()
        
        # Load the model
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        # Re-compile the model with a new optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),  # or any other optimizer
            loss='categorical_crossentropy',       # or your specific loss function
            metrics=['accuracy']
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the model in the newer Keras format
        model.save(path.with_suffix('.keras'))  # Save as .keras format

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [35]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-11-10 23:25:48,408: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-10 23:25:48,411: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-10 23:25:48,412: INFO: common: created directory at: artifacts]
[2024-11-10 23:25:48,413: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-11-10 23:25:48,414: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-11-10 23:25:48,417: INFO: common: created directory at: artifacts\training]
[2024-11-10 23:25:48,778: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.7579 - loss: 0.4993 - val_accuracy: 0.9219 - val_loss: 0.2863
Epoch 2/10
 1/19 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/

C:\Users\DIKSHANT PATEL\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 144ms/step - accuracy: 1.0000 - loss: 0.0932 - val_accuracy: 1.0000 - val_loss: 0.0234
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.9456 - loss: 0.1422 - val_accuracy: 0.9219 - val_loss: 0.2792
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 0.8750 - loss: 0.1611 - val_accuracy: 1.0000 - val_loss: 0.0348
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.9230 - loss: 0.2463 - val_accuracy: 0.9375 - val_loss: 0.3111
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - accuracy: 0.9375 - loss: 0.3194 - val_accuracy: 1.0000 - val_loss: 0.0338
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.9369 - loss: 0.1943 - val_accuracy: 0.9062 - val_loss: 0.3271
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 0.9375 - loss: 0.0790 - val_accuracy: 1.0000 - val_loss: 3.8574e-04
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.9512 - loss: 0.1481 - val_accuracy: 0.9062 - val_loss